# ginga-ml2-uncertainty-public — Turning Points PoC (Colab)

- 自分のJSONをアップロードして解析できます（アップしない場合はサンプルを使用）。
- 実行後、`assets/turning_points.png` が生成されます。


In [ ]:
# 1) clone repo (ColabでGitHubノートを開く場合、最初にcloneが安全です)
import os
repo = 'ginga-ml2-uncertainty-public'
if not os.path.exists(repo):
    !git clone -q https://github.com/Mokafe/ginga-ml2-uncertainty-public.git
%cd ginga-ml2-uncertainty-public


In [ ]:
# 2) install
!pip -q install -r requirements.txt


In [ ]:
# 3) upload your JSON (optional)
from google.colab import files
uploaded = files.upload()
user_json = next(iter(uploaded.keys())) if len(uploaded) else None
print('Uploaded JSON:', user_json)


In [ ]:
# 4) run
json_path = user_json if user_json else 'data/sample/sample_g_1_reconstructed_keep_last9.json'
from src.poc_turning_points import run
out_png = run(json_path=json_path, kimura2_txt='src/kimura2.txt', out_dir='assets', T=9, N=48)
print('Figure:', out_png)


In [ ]:
# 5) display figure
from PIL import Image
from IPython.display import display
display(Image.open('assets/turning_points.png'))


## GitHubに保存（Colab UI）
- `File > Save a copy in GitHub...` を選んで保存します。
- Repository: `Mokafe/ginga-ml2-uncertainty-public`
- File path: `notebooks/00_turning_points_colab.ipynb`
- Branch: `main`
